# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [16]:
import pandas as pd
import numpy as np
import gzip
import openpyxl
import pickle

from io import StringIO
import requests

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [17]:
# URL del archivo CSV comprimido en formato Gzip
url = 'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true'

# Descargar el archivo comprimido en formato Gzip
response = requests.get(url)
data = gzip.decompress(response.content).decode('utf-8')

# Crear un objeto StringIO para leer el CSV con pandas
loans = pd.read_csv(StringIO(data))

loans

/var/folders/7r/6s1q_b2x7rldv60cty2jvx440000gn/T/ipykernel_4389/2817855808.py:9: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv(StringIO(data))


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [18]:
column_types = loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [22]:
import pandas as pd
import requests
from io import BytesIO

# URL del archivo Excel
url = 'https://resources.lendingclub.com/LCDataDictionary.xlsx'

# Descargar el archivo
response = requests.get(url)

# Verificar si la descarga fue exitosa
if response.status_code == 200:
    # Leer el contenido del archivo descargado
    content = response.content
    # Convertir el contenido a un objeto BytesIO
    excel_data = BytesIO(content)
    # Leer el archivo Excel desde el objeto BytesIO
    datos_dict = pd.read_excel(excel_data)
    # Ahora puedes seguir utilizando los datos cargados
    datos_dict.columns = ['feature', 'description']
else:
    print("No se pudo descargar el archivo.")

# Hacer uso de los datos cargados


In [23]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [24]:
# Codigo guardar
with open('datos_dict.pkl', 'wb') as f:
    pickle.dump(datos_dict, f)

In [25]:
# Codigo para cargar
with open('datos_dict.pkl', 'rb') as f:
    loaded_df = pickle.load(f)

loaded_df

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [26]:
# Manejos de tipos 1
def convert_column_type(column):
    # Comprueba si todos los valores en la columna son cadenas (str)
    if all(isinstance(value, str) for value in column):
        return column  # Si todos son cadenas, devuelve la columna sin cambios

    # Intenta convertir la columna a tipo entero (int)
    try:
        column_as_int = pd.to_numeric(column, errors='raise', downcast='integer')
        # Comprueba si la columna convertida a entero es igual a la columna original
        if column_as_int.equals(column):
            return column_as_int  # Si son iguales, devuelve la columna convertida a entero
    except (ValueError, TypeError):
        pass  # Si hay un error al convertir, continúa con la siguiente conversión

    # Intenta convertir la columna a tipo flotante (float)
    try:
        column_as_float = pd.to_numeric(column, errors='raise', downcast='float')
        # Comprueba si la columna convertida a flotante es igual a la columna original
        if column_as_float.equals(column):
            return column_as_float  # Si son iguales, devuelve la columna convertida a flotante
    except (ValueError, TypeError):
        pass  # Si hay un error al convertir, devuelve la columna original

    # Si no se ha realizado ninguna conversión, devuelve la columna original
    return column


In [27]:
# Manejos de tipos 2
def convert_dataframe_types(dataframe):
    # Itera sobre cada nombre de columna en el DataFrame
    for column_name in dataframe.columns:
        # Convierte el tipo de la columna llamando a la función convert_column_type
        dataframe[column_name] = convert_column_type(dataframe[column_name])

    return dataframe  # Devuelve el DataFrame con las columnas convertidas


In [28]:
# column_types 
column_types = loans.dtypes
column_types

Unnamed: 0                 int64
id                         int64
member_id                float64
loan_amnt                float64
funded_amnt              float64
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 151, dtype: object

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [31]:
# Tu codigo aqui
imput_dict = {}
loans.replace('', np.nan, inplace=True) # reemplazamos los espacios en blanco por nans
def imputacion (dataframe):
    for column_name in dataframe.columns:
        if dataframe[column_name].dtype == 'int64':
            dataframe[column_name].fillna(int(dataframe[column_name].mean()), inplace=True)
            imput_dict[column_name] = {'estrategia': 'llenar con la media', 'valor': int(dataframe[column_name].mean())}
        # si la columna es int, la llenamos con la media para intentar no cambiar el como se distribuyen los datos
        if dataframe[column_name].dtype == 'float64':
            dataframe[column_name].fillna(dataframe[column_name].mean(), inplace=True)
            imput_dict[column_name] = {'estrategia': 'llenar con la media', 'valor': dataframe[column_name].mean()}
        # si la columna es float, la llenamos con la media para intentar no cambiar el como se distribuyen los datos
        if dataframe[column_name].dtype == 'object':
            dataframe[column_name].fillna(dataframe[column_name].mode(), inplace=True)
            imput_dict[column_name] = {'estrategia': 'llenar con la moda', 'valor': str(dataframe[column_name].mode())}
        # si la columna es de objetos, la llenamos con la moda para intentar seguir la tendencia de distribución de los datos
        if dataframe[column_name].dtype == 'string':
            dataframe[column_name].fillna(dataframe[column_name].mode(), inplace=True)
            imput_dict[column_name] = {'estrategia': 'llenar con la moda', 'valor': str(dataframe[column_name].mode())}
        # si la columna es de cadenas, la llenamos con la moda para intentar seguir la tendencia de distribución de los datos
        if dataframe[column_name].isnull().mean()>= .1:
            dataframe.drop(column_name, axis=1, inplace=True)
        # dropeamos las columnas que sean la mitad o más nans, para reducir las columnas vacias    
    return dataframe 
loans


,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
1,1,36805548,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
2,2,37842129,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,D1,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
3,3,37612354,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,D4,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
4,4,37662224,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,C3,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,B1,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
99996,99996,11396920,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,C1,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
99997,99997,8556176,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,E4,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116
99998,99998,24023408,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,F4,...,8046.616541,186.563135,Cash,N,Sep-2017,ACTIVE,Apr-2017,4768.376357,47.720519,8.265116


In [34]:
# TU codigo aqui
loans = imputacion(loans)

imput_dict

/var/folders/7r/6s1q_b2x7rldv60cty2jvx440000gn/T/ipykernel_4389/3600428040.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe[column_name].fillna(int(dataframe[column_name].mean()), inplace=True)
/var/folders/7r/6s1q_b2x7rldv60cty2jvx440000gn/T/ipykernel_4389/3600428040.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on wh

{'Unnamed: 0': {'estrategia': 'llenar con la media', 'valor': 49999},
 'id': {'estrategia': 'llenar con la media', 'valor': 30299954},
 'loan_amnt': {'estrategia': 'llenar con la media', 'valor': 14886.93},
 'funded_amnt': {'estrategia': 'llenar con la media', 'valor': 14886.93},
 'funded_amnt_inv': {'estrategia': 'llenar con la media', 'valor': 14883.9105},
 'term': {'estrategia': 'llenar con la moda',
  'valor': '0     36 months\nName: term, dtype: object'},
 'int_rate': {'estrategia': 'llenar con la media', 'valor': 13.2780734},
 'installment': {'estrategia': 'llenar con la media',
  'valor': 437.3318244000001},
 'grade': {'estrategia': 'llenar con la moda',
  'valor': '0    C\nName: grade, dtype: object'},
 'sub_grade': {'estrategia': 'llenar con la moda',
  'valor': '0    C1\nName: sub_grade, dtype: object'},
 'emp_title': {'estrategia': 'llenar con la moda',
  'valor': '0    Teacher\nName: emp_title, dtype: object'},
 'emp_length': {'estrategia': 'llenar con la moda',
  'valor': 

### Codigo para salvar y cargar JSONs

In [35]:
with open('imput_dict.json', 'w') as f:
    json.dump(imput_dict, f)

In [36]:
with open('imput_dict.json', 'r') as o:
    json_dict = json.load(o)
json_dict

{'Unnamed: 0': {'estrategia': 'llenar con la media', 'valor': 49999},
 'id': {'estrategia': 'llenar con la media', 'valor': 30299954},
 'loan_amnt': {'estrategia': 'llenar con la media', 'valor': 14886.93},
 'funded_amnt': {'estrategia': 'llenar con la media', 'valor': 14886.93},
 'funded_amnt_inv': {'estrategia': 'llenar con la media', 'valor': 14883.9105},
 'term': {'estrategia': 'llenar con la moda',
  'valor': '0     36 months\nName: term, dtype: object'},
 'int_rate': {'estrategia': 'llenar con la media', 'valor': 13.2780734},
 'installment': {'estrategia': 'llenar con la media',
  'valor': 437.3318244000001},
 'grade': {'estrategia': 'llenar con la moda',
  'valor': '0    C\nName: grade, dtype: object'},
 'sub_grade': {'estrategia': 'llenar con la moda',
  'valor': '0    C1\nName: sub_grade, dtype: object'},
 'emp_title': {'estrategia': 'llenar con la moda',
  'valor': '0    Teacher\nName: emp_title, dtype: object'},
 'emp_length': {'estrategia': 'llenar con la moda',
  'valor': 